In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Load a pre-trained object detection model (e.g., SSD MobileNet)
model = tf.saved_model.load('ssd_mobilenet_v2_fpnlite_320x320/saved_model')

# Load label map for Pokémon cards
def load_label_map(label_map_path):
    with open(label_map_path, 'r') as file:
        label_map = {int(line.split(':')[0]): line.split(':')[1].strip() for line in file.readlines()}
    return label_map

label_map = load_label_map('label_map.txt')  # Replace with your label map file

# Function to preprocess the input image
def preprocess_image(image):
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]
    return input_tensor

# Function to detect Pokémon cards
def detect_pokemon_cards(image, model, label_map):
    input_tensor = preprocess_image(image)
    detections = model(input_tensor)

    # Extract detection results
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy().astype(np.int32)
    detection_scores = detections['detection_scores'][0].numpy()

    # Filter detections based on a confidence threshold
    confidence_threshold = 0.5
    results = []
    for i in range(len(detection_scores)):
        if detection_scores[i] >= confidence_threshold:
            box = detection_boxes[i]
            class_id = detection_classes[i]
            label = label_map[class_id]
            score = detection_scores[i]
            results.append((box, label, score))
    return results

# Open the camera and detect Pokémon cards in real-time
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect Pokémon cards
    results = detect_pokemon_cards(frame, model, label_map)

    # Draw detection results on the frame
    for box, label, score in results:
        h, w, _ = frame.shape
        ymin, xmin, ymax, xmax = box
        (startX, startY, endX, endY) = (int(xmin * w), int(ymin * h), int(xmax * w), int(ymax * h))
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
        text = f"{label}: {score:.2f}"
        cv2.putText(frame, text, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Pokémon Card Detector', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

OSError: SavedModel file does not exist at: ssd_mobilenet_v2_fpnlite_320x320/saved_model\{saved_model.pbtxt|saved_model.pb}

: 